In [1]:
import sys
import os
sys.path.append("..")
from knool.remote_sensing.satellite import tles_for_asat
from knool.geodata_processor import geo_info
import pandas as pd
from skyfield.api import load
from datetime import datetime as dt
from datetime import timedelta
from skyfield.api import utc
import numpy as np
from osgeo import gdal, osr, ogr

In [2]:
##READ tle file. https://qiita.com/grinpeaceman/items/22f06714a98dbfacce88

tles = load.tle_file("../knool/remote_sensing/data/two_line_elements/sat40697.txt") # TLE読み込み
satellite=tles[0]
sgp4_model = satellite.model

print("衛星名: ", satellite.name)
print("元期: ", satellite.epoch.utc_jpl())
print("衛星番号: ", sgp4_model.satnum)
print("エポック年: ", sgp4_model.epochyr)
print("エポック日: ", sgp4_model.epochdays)
print("エポックのユリウス日: ", sgp4_model.jdsatepoch)
print("ndot: ", sgp4_model.ndot)
print("nddot: ", sgp4_model.nddot)
print("弾道抗力係数 B*: ", sgp4_model.bstar)
print("軌道傾斜角[rad]: ", sgp4_model.inclo)
print("昇交点赤経[rad]: ", sgp4_model.nodeo)
print("離心率: ", sgp4_model.ecco)
print("近地点引数[rad]: ", sgp4_model.argpo)
print("平均近点角[rad]: ", sgp4_model.mo)
print("平均運動[rad/min]: ", sgp4_model.no_kozai)

衛星名:  None
元期:  A.D. 2015-Jun-23 03:50:23.3849 UTC
衛星番号:  40697
エポック年:  15
エポック日:  174.15999288
エポックのユリウス日:  2457196.5
ndot:  -1.333237623051224e-12
nddot:  0.0
弾道抗力係数 B*:  0.0
軌道傾斜角[rad]:  1.7206276070908537
昇交点赤経[rad]:  4.337586504835915
離心率:  7.98e-05
近地点引数[rad]:  4.953066393576204
平均近点角[rad]:  2.5043519997356394
平均運動[rad/min]:  0.06244205289471824


In [3]:
#get lat,lon,elevation from a tle
ts = load.timescale()  # skyfiled上で時刻を扱うモジュール
tle=tles[0]
date1 = dt(2018, 2, 1, 12, 15, 30, 2000, tzinfo=utc)
date1_sf=ts.from_datetime(date1)

geocentric=tle.at(date1_sf) #get xyz
subpoint = geocentric.subpoint()
lat = subpoint.latitude.degrees
lon = subpoint.longitude.degrees
ele = subpoint.elevation.m
print(geocentric.velocity)
print(geocentric.position)
print(geocentric.xyz)
print(lat,lon,ele)

[-0.00141461  0.00399833  0.00070632] au/day
[-9.51255273e-06  4.88624241e-06 -4.67440961e-05] au
[-9.51255273e-06  4.88624241e-06 -4.67440961e-05] au
-77.27650275778588 -162.68188667619506 815689.5183005581


In [4]:
#convert lla/ecef from ecef/lla using our package.
print(lat,lon,ele)
x,y,z=geo_info.convert_lla_to_ecef(lat,lon,ele)
print(x,y,z) # meter
lat,lon,ele=geo_info.convert_ecef_to_lla(x,y,z)
print(lat,lon,ele)

-77.27650275778588 -162.68188667619506 815689.5183005581
-1516878.3505331383 -472981.10875354544 -6995304.199162432
-77.2765027607371 -162.68188667619506 815689.519945832


In [5]:
#create a kml file of a orbit line
infile=r"../knool/remote_sensing/data/two_line_elements/sat27424.txt"
test=tles_for_asat(infile)
test.set_period("2011-06-30 10:00:00","2011-07-02 11:40:00")
test.calc_positions_between("2011-07-01 10:00:00","2011-07-01 11:40:00",1)
test.export("../../test_data/output/test.kml")

d:\プログラム\2022\kn-general_analysis_tools\notebook\..\knool\remote_sensing\satellite.py:40: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  row1=df.loc[df.index==df.index.unique()[df.index.unique().get_loc(pd.to_datetime(date), method='nearest')]]


In [6]:
#Creates a kml file of a line that are a specified distance away from the orbital line.
test.calc_buff_positions_between("2011-07-01 10:00:00","2011-07-01 11:40:00",1,1000000,ori="left")
test.export("../../test_data/output/test2.kml")

In [7]:
#Creates a kml file of a polygon
test.calc_buff_area_between("2011-07-01 10:00:00","2011-07-01 11:00:00",0.5,1000000,ori="left")
test.export("../../test_data/output/test3.kml")

In [8]:
#create a kml file of a orbit line
test.calc_positions_faster_between("2011-07-01 10:00:00","2011-07-01 11:40:00",1)
test.export("../../test_data/output/test4.kml")

d:\プログラム\2022\kn-general_analysis_tools\notebook\..\knool\remote_sensing\satellite.py:65: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  row1=df.loc[df.index==df.index.unique()[df.index.unique().get_loc(pd.to_datetime(mdt), method='nearest')]]


In [25]:
# #Creates a kml file of a line that are a specified distance away from the orbital line.
# test.calc_buff_positions_faster_between("2011-07-01 10:00:00","2011-07-01 11:40:00",1,1000000,ori="left")
# test.export("../../test_data/output/test2.kml")


def convert_enu_to_ecef(x0, y0, z0, lat0, lon0, h0, x, y, z):

    if type(lat0).__module__ != "numpy":
        proc=2
    elif isinstance(lat0.shape,tuple):
        proc=2
    else:
        proc=1

    if proc==1:
        
        px=np.radians([90 for i in range(x0.shape[0])])
        py=np.radians(90-lat0)
        pz=np.radians(lon0)
        pz2=np.radians([90 for i in range(x0.shape[0])])
        p2=np.array([x,y,z])
        p0=np.array([x0,y0,z0])
        dp=p1-p0
        
        Ry = np.array([[np.cos(py), np.zeros(x0.shape), -np.sin(py)],
                    [np.zeros(x0.shape), np.ones(x0.shape), np.zeros(x0.shape)],
                    [np.sin(py), np.zeros(x0.shape), np.cos(py)]]).transpose([2,0,1])
        
        Rz = np.array([[np.cos(pz), np.sin(pz), np.zeros(x0.shape)],
                    [-np.sin(pz), np.cos(pz), np.zeros(x0.shape)],
                    [np.zeros(x0.shape), np.zeros(x0.shape), np.ones(x0.shape)]]).transpose([2,0,1])
        
        Rz2 = np.array([[np.cos(pz2), np.sin(pz2), np.zeros(x0.shape)],
                    [-np.sin(pz2), np.cos(pz2), np.zeros(x0.shape)],
                    [np.zeros(x0.shape), np.zeros(x0.shape), np.ones(x0.shape)]]).transpose([2,0,1])
        
        R = np.linalg.inv(np.matmul(np.matmul(Rz2,Ry),Rz))
        print(R.shape,dp.shape)
        result= R.dot(dp) + p0
        x=np.diag(result[:,0,:])
        y=np.diag(result[:,1,:])
        z=np.diag(result[:,2,:])

    else:
        px=np.radians(90)
        py=np.radians(90-lat0)
        pz=np.radians(lon0)
        pz2=np.radians(90)
        p2=np.array([x,y,z])
        p0=np.array([x0,y0,z0])

        Ry = np.array([[np.cos(py), 0, -np.sin(py)],
                    [0, 1, 0],
                    [np.sin(py), 0, np.cos(py)]])
        
        Rz = np.array([[np.cos(pz), np.sin(pz), 0],
                    [-np.sin(pz), np.cos(pz), 0],
                    [0, 0, 1]])
        
        Rz2 = np.array([[np.cos(pz2), np.sin(pz2), 0],
                    [-np.sin(pz2), np.cos(pz2), 0],
                    [0, 0, 1]])

        R = np.linalg.inv(Rz2.dot(Ry).dot(Rz))
        p1 = R.dot(p2) + p0
                
    return p1


def calc_line_buffer_point(lat0,lon0,h0,lat,lon,h,distance,ori="right"):
    R=6373000.0
    x0, y0, z0 = geo_info.convert_lla_to_ecef(lat0, lon0, h0, a=R, b=R)
    x, y, z = geo_info.convert_lla_to_ecef(lat, lon, h, a=R, b=R)    
    p, q, r = geo_info.convert_ecef_to_enu(x0, y0, z0, lat0, lon0 , h0, x, y, z)
    print(p.shape,q.shape,r.shape)
    print(x.shape,y.shape,z.shape)
    print(x,y,z)
    print(p,q,r)
    sign=1
    if ori=="left" : sign=-1
    
    theta=distance/R
    tan_th=np.tan(theta)
    distance2=R*tan_th
    coef=1/(q*q/(p*p)+1)
    print(coef)
    print(coef.shape)
    q2 = np.sqrt(distance2**2 * coef)
    q2= sign*q2
    p2 = -q*q2/p
    r2 = 0

    buff_ecef=convert_enu_to_ecef(x0, y0, z0, lat0, lon0 , h0, p2, q2, r2)
    lat,lon,h=geo_info.convert_ecef_to_lla(*buff_ecef, a=R, b=R)
    
    return lat, lon, h

lat1=np.array([0,1])
lon1=np.array([20,30])
lat2=np.array([0.1,1.1])
lon2=np.array([20.01,30.01])
h1=np.array([0,0])
h2=np.array([0,0])

#lat1=0;lon1=20;h1=0;lat2=0.1;lon2=20.01;h2=0
#lat1=1;lon1=30;h1=0;lat2=1.1;lon2=30.01;h2=0
test=calc_line_buffer_point(lat1,lon1,h1,lat2,lon2,h2,100000,ori="right")
print(test)


(2,) (2,) (2,)
(2,) (2,) (2,)
[5988271.43195243 5517606.6496463 ] [2180736.23729006 3186875.81923829] [ 11122.97767587 122345.30041322]
[array([1112.29663252, 1097.10074162])
 array([1059.78695452, 1112.09334355])] [array([11122.97767587, 11118.08118891])
 array([11121.11418841, 11122.9793696 ])] [array([ -9.80369781, 377.58830589]) array([-393.93983351,   -9.80366529])]
[array([0.00990097, 0.00964328]) array([0.00899942, 0.00989738])]
(2,)


d:\プログラム\2022\kn-general_analysis_tools\notebook\..\knool\geodata_processor\geo_info.py:228: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Ry = np.array([[np.cos(py), 0, -np.sin(py)],
d:\プログラム\2022\kn-general_analysis_tools\notebook\..\knool\geodata_processor\geo_info.py:232: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Rz = np.array([[np.cos(pz), np.sin(pz), 0],


TypeError: loop of ufunc does not support argument 0 of type numpy.ndarray which has no callable sqrt method

In [10]:
(0.08945403663565143, 19.10542164823447, 784.6403572112322)


NameError: name 'proc' is not defined